## **Desafio Técnico**

Para as questões de 1 a 5 foram utilizadas as tabelas **chamados_marco_abril** e **bairros**, e para as questões de 6 a 10, as tabelas **chamados_2021_2022_2023** e **eventos**. Pelo conjunto de dados Chamado 1746 ter mais de **3 GB**, selecionou-se subconjuntos menores (**chamados_marco_abril** e **chamados_2021_2022_2023**) para servirem como base de dados para as consultas. Nota-se que os scripts SQLs utilizados para gerar estes conjuntos, buscam incluir dados a mais que o necessário para as questões, com o objetivo de simular o uso de um conjunto arbitrário, deixando todas transformações necessárias para responder as perguntas, a cargo do código.

Sobre as soluções, utilizou-se ***list comprehension*** e outras formas de manipulação de dados não usuais, com o objetivo de minimizar a complexidade assintótica do código. Cada solução contém comentários para ilustração das operações.

Recomenda-se realizar a execução deste Notebook na plataforma **Google Colab**. As instruções para execução estão em [README.md](README.md).

In [ ]:
!pip install basedosdados

import basedosdados as bd
import pandas as pd
import datetime

In [2]:
query_chamados_marco_abril = """
    SELECT tipo, id_bairro, data_inicio, data_particao FROM `datario.administracao_servicos_publicos.chamado_1746`
      WHERE data_particao BETWEEN '2023-03-01' AND '2023-04-01'"""
chamados_marco_abril = bd.read_sql(query_chamados_marco_abril, billing_project_id="emd-desafio-data-scientist")

Downloading: 100%|██████████| 149781/149781 [00:10<00:00, 14159.92rows/s]


In [3]:
query_bairros = "SELECT id_bairro, nome, subprefeitura FROM `datario.dados_mestres.bairro`"
bairros = bd.read_sql(query_bairros, billing_project_id="emd-desafio-data-scientist")

Downloading: 100%|██████████| 164/164 [00:00<00:00, 440.31rows/s]


In [4]:
query_chamados_2021_2022_2023 = """
    SELECT tipo, subtipo, id_bairro, data_inicio, data_particao FROM `datario.administracao_servicos_publicos.chamado_1746`
      WHERE data_particao BETWEEN '2021-12-01' AND '2023-12-01'"""
chamados_2021_2022_2023 = bd.read_sql(query_chamados_2021_2022_2023, billing_project_id="emd-desafio-data-scientist")

Downloading: 100%|██████████| 1678541/1678541 [02:29<00:00, 11211.15rows/s]


In [5]:
query_eventos = "SELECT data_inicial, data_final, evento FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`"
eventos = bd.read_sql(query_eventos, billing_project_id="emd-desafio-data-scientist")

Downloading: 100%|██████████| 4/4 [00:00<00:00, 11.18rows/s]


In [6]:
# 1. Quantos chamados foram abertos no dia 01/04/2023?

chamados_marco_abril["data_particao"].astype("datetime64")

particao_abril = datetime.date(2023, 4, 1)

# Seleção dos chamados com data_particao == "2023-04-01"
chamados_abril = chamados_marco_abril[chamados_marco_abril["data_particao"] == particao_abril]

dia_alvo = datetime.date(2023, 4, 1)

# Seleção dos chamados com data_inicio == "2023-04-01"
chamados_diaalvo = chamados_abril[(chamados_abril["data_inicio"].dt.date).values == dia_alvo]

numero_chamados_diaalvo = len(chamados_diaalvo)

print("1. Quantos chamados foram abertos no dia 01/04/2023?")
print(numero_chamados_diaalvo)

1. Quantos chamados foram abertos no dia 01/04/2023?
73


In [16]:
# 2. Qual o tipo de chamado que mais teve chamados abertos no dia 01/04/2023?

tipo_maior_soma = ""
maior_soma = 0

# Tipos de chamados feitos em data_inicio == "2023-04-01"
tipos_chamados_diaalvo = chamados_diaalvo["tipo"].unique()

# Procura do tipo com mais chamados no dia alvo
for tipo in tipos_chamados_diaalvo:
  soma = (chamados_diaalvo["tipo"] == tipo).sum()

  if soma > maior_soma:
    maior_soma = soma
    tipo_maior_soma = tipo

print("2. Qual o tipo de chamado que mais teve chamados abertos no dia 01/04/2023?")
print(tipo_maior_soma,"(",maior_soma,")")

2. Qual o tipo de chamado que mais teve chamados abertos no dia 01/04/2023?
Poluição sonora ( 24 )


In [8]:
# 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

# Lista de Ids dos bairros da cidade
ids_bairros = bairros["id_bairro"]

# Construção de um dicionário com o Id de um bairro como chave, e o número de chamados
# feitos no mesmo (no dia alvo) como valor.
rank_bairros = {
    id_bairro: (chamados_diaalvo["id_bairro"] == id_bairro).sum()
    for id_bairro
    in ids_bairros
    }

# Ordenação das entradas do dicionário por número de chamados (bairro[1]).
# bairro[0] equivale ao Id do bairro.
rank_bairros = sorted(rank_bairros.items(), key = lambda bairro: bairro[1])

# Percorre os três últimos elementos do dicionário (rank_bairros[-3:]) para formar
# tuplas do tipo (nome_bairro, numero_chamados).
# bairros[bairros["id_bairro"] == bairro[0]] seleciona a linha de um bairro por
# comparação de Ids.
# .iat[0, 1] seleciona o nome do bairro.
top_3_bairros = [
    (bairros[bairros["id_bairro"] == bairro[0]].iat[0, 1], bairro[1])
    for bairro
    in rank_bairros[-3:]
    ]

print("3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?")
print(top_3_bairros)

3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
[('Leblon', 6), ('Campo Grande', 6), ('Engenho de Dentro', 8)]


In [9]:
# 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

# Lista dos nomes das prefeituras
subprefeituras = bairros["subprefeitura"].unique()

# Dicionário (1) com o Id de um bairro como chave e o nome da sua subprefeitura como valor
relacao_id_subprefeitura = bairros.set_index("id_bairro")["subprefeitura"].to_dict()

# Inicializa dicionário (2). Terá nomes de subprefeituras como chave e o número de
# chamados feitos nas mesmas (no dia alvo) como valor.
contagem_chamados_subprefeitura_diaalvo = {
    subprefeitura: 0
    for subprefeitura
    in subprefeituras
}

# Percorre chamados feitos no dia alvo
for i in range(numero_chamados_diaalvo):
  id_bairro = chamados_diaalvo.iat[i, 1]

  if id_bairro == None:
    continue

  # Utiliza dicionário (1) para recuperar o nome da subprefeitura do bairro
  subprefeitura = relacao_id_subprefeitura[id_bairro]

  # Utiliza dicionário (2) para incrementar a contagem da subprefeitura em questão
  contagem_chamados_subprefeitura_diaalvo[subprefeitura] += 1

# Ordenação do dicionário (2) por número de chamados
contagem_chamados_subprefeitura_diaalvo = sorted(contagem_chamados_subprefeitura_diaalvo.items(), key = lambda contagem : contagem[1])

print("4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?")
print(contagem_chamados_subprefeitura_diaalvo[-1])

4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
('Zona Norte', 25)


In [10]:
# 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

num_chamados_sem_bairro = 0

# Percorre os chamados feitos no dia alvo, à procura de um chamado se bairro associado
for i in range(numero_chamados_diaalvo):
  id_bairro = chamados_diaalvo.iat[i, 1]

  if id_bairro == None:
    num_chamados_sem_bairro += 1
    #print(chamados_diaalvo.iloc[i][0:2])

print("5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?")
print("\nNúmero de chamados sem bairro:", num_chamados_sem_bairro)

5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Número de chamados sem bairro: 1


In [18]:
# 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

# Lista das partições existentes do início de 2022 até o fim de 2023 (todas no formato YYYY-MM-01)
data_particoes = pd.date_range(start="01/01/2022",end="31/12/2023", freq="MS").date

chamados_2021_2022_2023["data_particao"].astype("datetime64")

# Contagem do número de chamados feitos em cada partição.
# chamados_2021_2022_2023["data_particao"] == particao identifica as linhas que contém chamados com data_particao = particao
# com o valor True, e as demais com False.
# (chamados_2021_2022_2023[filtro_anterior]["subtipo"] == "Perturbação do sossego").values.tolist()
# seleciona as linhas do DataFrame correspondentes aos valores True, e identifica as linhas com chamados do subtipo "Perturbação do sossego"
# com valor True, e as demais com False.
# sum(operacoes_acima) resulta no número de chamados numa particao. Valores True somam + 1, e False valem 0.
contagem_chamados_particoes = [
    sum((chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]["subtipo"] == "Perturbação do sossego").values.tolist())
    for particao
    in data_particoes
]

print('6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?')
print(sum(contagem_chamados_particoes))

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
42408


In [12]:
# 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

dados_reveillon = eventos.loc[eventos["evento"] == "Reveillon"]
dados_carnaval = eventos.loc[eventos["evento"] == "Carnaval"]
dados_rockrio = eventos.loc[eventos["evento"] == "Rock in Rio"]

periodo_reveillon = []

# Este e os dois loops abaixo registram os períodos de cada evento. Por exemplo, o Reveillon e o Carnaval
# apresentam um só período cada (Reveillon: 2022-12-30 a 2023-01-01, Carnaval: 2023-02-18 a 2023-02-21).
# Já o Rock in Rio apresenta dois: 2022-09-08 a 2022-09-11, e 2022-09-02 a 2022-09-04.
for i in range(len(dados_reveillon)):
  periodo_reveillon.append(pd.date_range(dados_reveillon.iat[i, 0], dados_reveillon.iat[i, 1]).date.tolist())

periodo_carnaval = []
for i in range(len(dados_carnaval)):
  periodo_carnaval.append(pd.date_range(dados_carnaval.iat[i, 0], dados_carnaval.iat[i, 1]).date.tolist())

periodo_rockrio = []
for i in range(len(dados_rockrio)):
  periodo_rockrio.append(pd.date_range(dados_rockrio.iat[i, 0], dados_rockrio.iat[i, 1]).date.tolist())

# Este e os dois loops abaixo definem as partições necessárias para as consultas de
# cada evento. Por exemplo, o Carnaval precisará da partição 2023-02-01, e o Rock in Rio
# da partição 2022-09-01. Já o Reveillon, fará uso das partições 2022-12-01 e 2023-01-01.
set_particoes_reveillon = set()
for i in range(len(periodo_reveillon)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_reveillon[i]
  ]
  set_particoes_reveillon.update(particoes_periodo)

list_particoes_reveillon = list(set_particoes_reveillon)

set_particoes_carnaval = set()
for i in range(len(periodo_carnaval)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_carnaval[i]
  ]
  set_particoes_carnaval.update(particoes_periodo)

list_particoes_carnaval = list(set_particoes_carnaval)

set_particoes_rockrio = set()
for i in range(len(periodo_rockrio)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_rockrio[i]
  ]
  set_particoes_rockrio.update(particoes_periodo)

list_particoes_rockrio = list(set_particoes_rockrio)

# Armazenará o resultado da questão: chamados do subtipo "Perturbação do sossego" feitos nos eventos
selecao_chamados_subtipo_eventos = []

# Este e os dois loops de mesma identação abaixo, adicionam ao resultado, os chamados com o determinado
# subtipo em cada evento.
for particao in list_particoes_reveillon:

  # Seleciona os chamados de uma partição do evento
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  # Um evento tem um ou mais períodos, e cada período tem um ou mais dias.
  for periodo in periodo_reveillon:
    for data in periodo:
      # Armazena os chamados feitos num dia do evento.
      dados_data = dados_particao[dados_particao["data_inicio"].dt.date == data]

      # Adiciona ao resultado, os chamados do determinado subtipo feitos neste dia
      selecao_chamados_subtipo_eventos.extend(dados_data[dados_data["subtipo"] == "Perturbação do sossego"].values.tolist())

for particao in list_particoes_carnaval:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  for periodo in periodo_carnaval:
    for data in periodo:
      dados_data = dados_particao[dados_particao["data_inicio"].dt.date == data]
      selecao_chamados_subtipo_eventos.extend(dados_data[dados_data["subtipo"] == "Perturbação do sossego"].values.tolist())

for particao in list_particoes_rockrio:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  for periodo in periodo_rockrio:
    for data in periodo:
      dados_data = dados_particao[dados_particao["data_inicio"].dt.date == data]
      selecao_chamados_subtipo_eventos.extend(dados_data[dados_data["subtipo"] == "Perturbação do sossego"].values.tolist())

df_selecao_chamados_subtipo_eventos = pd.DataFrame(selecao_chamados_subtipo_eventos, columns=["tipo", "subtipo", "id_bairro", "data_inicio", "data_particao"])

print("7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).")
print(df_selecao_chamados_subtipo_eventos.head())
print("Número de linhas: ", df_selecao_chamados_subtipo_eventos.shape[0])

7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
              tipo                 subtipo id_bairro         data_inicio  \
0  Poluição sonora  Perturbação do sossego        20 2022-12-30 00:42:27   
1  Poluição sonora  Perturbação do sossego        25 2022-12-30 01:41:32   
2  Poluição sonora  Perturbação do sossego        45 2022-12-30 22:53:52   
3  Poluição sonora  Perturbação do sossego        83 2022-12-30 16:48:54   
4  Poluição sonora  Perturbação do sossego       106 2022-12-30 22:51:00   

  data_particao  
0    2022-12-01  
1    2022-12-01  
2    2022-12-01  
3    2022-12-01  
4    2022-12-01  
Número de linhas:  1212


In [19]:
# 8. Quantos chamados desse subtipo foram abertos em cada evento?
# 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

dados_reveillon = eventos.loc[eventos["evento"] == "Reveillon"]
dados_carnaval = eventos.loc[eventos["evento"] == "Carnaval"]
dados_rockrio = eventos.loc[eventos["evento"] == "Rock in Rio"]

# Asim como na questão 7, este e os dois loops abaixo definem o período dos eventos.
# Além disso, para questão 9 faz-se a contagem dos dias nestes períodos.
periodo_reveillon = []
num_dias_reveillon = 0
for i in range(len(dados_reveillon)):
  datas_reveillon = pd.date_range(dados_reveillon.iat[i, 0], dados_reveillon.iat[i, 1]).date.tolist()
  periodo_reveillon.append(datas_reveillon)
  num_dias_reveillon += len(datas_reveillon)

num_dias_carnaval = 0
periodo_carnaval = []
for i in range(len(dados_carnaval)):
  datas_carnaval = pd.date_range(dados_carnaval.iat[i, 0], dados_carnaval.iat[i, 1]).date.tolist()
  periodo_carnaval.append(datas_carnaval)
  num_dias_carnaval += len(datas_carnaval)

num_dias_rockrio = 0
periodo_rockrio = []
for i in range(len(dados_rockrio)):
  datas_rockrio = pd.date_range(dados_rockrio.iat[i, 0], dados_rockrio.iat[i, 1]).date.tolist()
  periodo_rockrio.append(datas_rockrio)
  num_dias_rockrio += len(datas_rockrio)

# Definição das partições para cada evento
set_particoes_reveillon = set()
for i in range(len(periodo_reveillon)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_reveillon[i]
  ]
  set_particoes_reveillon.update(particoes_periodo)

list_particoes_reveillon = list(set_particoes_reveillon)

set_particoes_carnaval = set()
for i in range(len(periodo_carnaval)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_carnaval[i]
  ]
  set_particoes_carnaval.update(particoes_periodo)

list_particoes_carnaval = list(set_particoes_carnaval)

set_particoes_rockrio = set()
for i in range(len(periodo_rockrio)):
  particoes_periodo = [
      pd.to_datetime(data).to_numpy().astype('datetime64[M]')
      for data
      in periodo_rockrio[i]
  ]
  set_particoes_rockrio.update(particoes_periodo)

list_particoes_rockrio = list(set_particoes_rockrio)

# Este e os dois loops abaixo, de mesma identação, fazem a soma dos chamados feitos em cada evento,
# do determinado subtipo.
soma_reveillon = 0
for particao in list_particoes_reveillon:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  for periodo in periodo_reveillon:
    chamados_reveillon = [
        # Nota-se que abaixo é utilizado a mesma lógica explicada na questão 6
        sum((dados_particao[dados_particao["data_inicio"].dt.date == data]["subtipo"] == "Perturbação do sossego").values.tolist())
        for data
        in periodo
    ]

    # Adiciona-se à soma total, a soma do número de chamados feitos em cada período do evento
    soma_reveillon += sum(chamados_reveillon)

soma_carnaval = 0
for particao in list_particoes_carnaval:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  for periodo in periodo_carnaval:
    chamados_carnaval = [
        sum((dados_particao[dados_particao["data_inicio"].dt.date == data]["subtipo"] == "Perturbação do sossego").values.tolist())
        for data
        in periodo
    ]

    soma_carnaval += sum(chamados_carnaval)

soma_rockrio = 0
for particao in list_particoes_rockrio:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]

  for periodo in periodo_rockrio:
    chamados_rockrio = [
        sum((dados_particao[dados_particao["data_inicio"].dt.date == data]["subtipo"] == "Perturbação do sossego").values.tolist())
        for data
        in periodo
    ]

    soma_rockrio += sum(chamados_rockrio)

print("8. Quantos chamados desse subtipo foram abertos em cada evento?")
print("Número de chamados no Reveillon: ", soma_reveillon)
print("Número de chamados no Carnaval: ", soma_carnaval)
print("Número de chamados no Rock in Rio: ", soma_rockrio)
print("\n")
print("9. Qual evento teve a maior média diária de chamados abertos desse subtipo?")
print("Média Reveillon: ", soma_reveillon/num_dias_reveillon)
print("Média Carnaval: ", soma_carnaval/num_dias_carnaval)
print("Média Rock in Rio: ", soma_rockrio/num_dias_rockrio)

8. Quantos chamados desse subtipo foram abertos em cada evento?
Número de chamados no Reveillon:  137
Número de chamados no Carnaval:  241
Número de chamados no Rock in Rio:  834
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
Média Reveillon:  45.666666666666664
Média Carnaval:  60.25
Média Rock in Rio:  119.14285714285714


In [20]:
# 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

data_particoes = pd.date_range(start="01/01/2022",end="31/12/2023", freq="MS").date

soma_chamados = 0
numero_dias = 0
for particao in data_particoes:
  dados_particao = chamados_2021_2022_2023[chamados_2021_2022_2023["data_particao"] == particao]
  dias = dados_particao["data_inicio"].dt.date.unique()

  chamados_subtipo_dia_mes = [
      sum((dados_particao[dados_particao["data_inicio"].dt.date == dia]["subtipo"] == "Perturbação do sossego").values.tolist()
      )
      for dia
      in dias
  ]

  soma_chamados += sum(chamados_subtipo_dia_mes)

  # Define-se o número de dias válidos (contidos na partição e com o determinado subtipo) através da contagem de valores diferentes
  # de 0 (len(lista) - lista.count(0)). chamados_subtipo_dia_mes contém o número de chamados feitos em cada dia de uma partição.
  # len(chamados_subtipo_dia_mes) retorna o número de dias na partição.
  numero_dias += len(chamados_subtipo_dia_mes) - chamados_subtipo_dia_mes.count(0)

print("10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.")
print("Média diária do subtipo de 01/01/2022 até 31/12/2023: ", soma_chamados/numero_dias)

10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.
Média diária do subtipo de 01/01/2022 até 31/12/2023:  63.20119225037258
